The code below was run on the Databricks notebook, following on from the code found in the dataframe cleaning notebook

In [ ]:
# Import necessary library for window functions
from pyspark.sql.window import Window

# Joining DataFrames df_pin and df_geo based on 'ind' column
pin_geo = df_pin.join(df_geo, df_pin.ind == df_geo.ind)

# Joining DataFrames df_pin and df_user based on 'ind' column and creating a temporary SQL view named 'category_age'
df_pin.join(df_user, df_pin.ind == df_user.ind).createOrReplaceTempView("category_age")

# SQL query to create an 'age_group' column based on specified age ranges
pin_user_age_group = spark.sql(
    "SELECT CASE \
        WHEN age BETWEEN 18 AND 24 THEN '18-24' \
        WHEN age BETWEEN 25 AND 35 THEN '25-35' \
        WHEN age BETWEEN 36 AND 50 THEN '36-50' \
        WHEN age > 50 THEN '50+' \
        END AS age_group, * FROM category_age")


## Identifying the Most Popular Category in Each Country


In [ ]:
# create partition by country and order by category_count descending
windowCountryByCatCount = Window.partitionBy("country").orderBy(col("category_count").desc())
# find the most popular category in each country
pin_geo.groupBy("country", "category") \
.agg(count("category") \
.alias("category_count")) \
.withColumn("rank", row_number().over(windowCountryByCatCount)) \
.filter(col("rank") == 1) \
.drop("rank") \
.show()
 # Filter to retain only the top-ranked category within each country
# .drop("rank") \  # Drop the 'rank' column, as it was only used for filtering
# .show()  # Display the result DataFrame containing the most popular category in each country


## Determining the Most Popular Category Each Year


In [ ]:
# create partition by year and order by category_count descending
windowYearByCatCount = Window.partitionBy("post_year").orderBy(col("category_count").desc())
# find which was the most popular category each year between 2018 and 2022
pin_geo.withColumn("post_year", year("timestamp")) \
.filter(col("post_year") >= 2018) \
.filter(col("post_year") <= 2022) \
.groupBy("post_year", "category") \
.agg(count("category").alias("category_count")) \
.withColumn("rank", row_number().over(windowYearByCatCount)) \
.filter(col("rank") == 1) \
.drop("rank") \
.show()

# Filter the DataFrame 'pin_geo' to include only records between the years 2018 and 2022
# Group by 'post_year' and 'category', count occurrences of each category within the specified year range
# Alias the count as 'category_count', then use window functions to rank the categories within each year

## Finding the User with the Most Followers in Each Country

Based on the prior query, Identify the Country Hosting the User with the Highest Follower Count


In [ ]:
# create partition by country and order by follower_count descending
windowCountryByFollowers = Window.partitionBy("country").orderBy(col("follower_count").desc())

# find the user with the most followers in each country
max_followers_by_country = \
    df_pin.join(df_geo, df_pin.ind == df_geo.ind) \
    .withColumn("rank", row_number().over(windowCountryByFollowers)) \
    .filter(col("rank") == 1) \
    .select("country", "poster_name", "follower_count")

# get highest number of followers from all countries
max_followers_all_countries = max_followers_by_country.select(max("follower_count")).collect()[0][0]

# find the country with the user with most followers
country_with_max_followers = \
    max_followers_by_country \
    .select("*") \
    .where(col("follower_count") == max_followers_all_countries)

max_followers_by_country.show()
country_with_max_followers.show()

## Determine the Most Popular Category Among Different Age Groups


In [ ]:
windowAgeGroup = Window.partitionBy("age_group").orderBy(col("category_count").desc())
pin_user_age_group.groupBy("age_group", "category") \
.agg(count("category").alias("category_count")) \
.withColumn("rank", row_number().over(windowAgeGroup)) \
.filter(col("rank") == 1) \
.drop("rank") \
.show()
# Create a window partitioned by 'age_group' and ordered by 'category_count' in descending order
# Grouping the DataFrame 'pin_user_age_group' by 'age_group' and 'category'
# Counting occurrences of each category within different age groups
# Ranks the categories based on their count within each age group
# Filters to retain only the top-ranked category within each age group
# Displays the result DataFrame containing the most popular category for different age groups

## Find the median follower count for different age groups

In [ ]:
# find the median follower count for different age groups
pin_user_age_group \
.select("user_name", "date_joined", "age_group", "follower_count") \
.distinct() \
.groupBy("age_group") \
.agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")) \
.orderBy("age_group") \
.show()
# Display the results
# Select specific columns for analysis: 'user_name', 'date_joined', 'age_group', 'follower_count'
# Ensure distinct records based on the selected columns
# Group the DataFrame by 'age_group'
# Calculate the median follower count for different age groups
# Order the results by 'age_group'
# Display the calculated median follower count for different age groups

## Find out how many users joined each year

In [ ]:
# find out how many users joined each year
df_user.withColumn("post_year", year("date_joined")) \
.drop("ind") \
.distinct() \
.groupBy("post_year") \
.agg(count("user_name").alias("number_users_joined")) \
.orderBy("post_year") \
.show()